In [1]:
import jsonschema
import json

def validate_json_file(common_schema_file, object_schema_file, json_data_file):
    """
    Function that, given a common-definitions schema and a specific data schema (both written in JSON),
        validates a third JSON file with the first two. For further details, go to: 
            https://github.com/EbiEga/ega-metadata-schema/tree/main/schemas
    """
    # Load the common-definitions JSON schema
    with open(common_schema_file, "r") as f:
        common_schema = json.load(f)
        
    # Load the specific schema
    with open(object_schema_file, "r") as f:
        object_schema = json.load(f)
        
    # Load the JSON data file
    with open(json_data_file, "r") as f:
        data_json = json.load(f)
    
    # We create the reference dictionary linking $id and $ref between files
    schema_store = {
        common_schema['$id'] : common_schema,
        object_schema['$id'] : object_schema
    }
    resolver = jsonschema.RefResolver.from_schema(common_schema, store=schema_store)
    validator = jsonschema.Draft7Validator(object_schema, resolver=resolver)

    # We try to validate the given JSON data file against the given schemas
    try:
        validator.validate(data_json)
        print(f"- Correctly validated file: {json_data_file}\n")
        
    except jsonschema.exceptions.ValidationError as error_message:
        print(f"-/- Validation error found at file: {json_data_file}")
        print(f"\tCause: {error_message.message}")

In [2]:
import os

# We iterate over all json files in the current directory and use the above defined
#     function to validate them. 
for testing_file in os.listdir("./"):
    if testing_file.endswith(".json"):
        # We extract the metadata object to use the correct object schema
        object_of_file = testing_file[:testing_file.find("-")]
        validate_json_file(common_schema_file = "../EGA.common-definitions.json", 
                           object_schema_file = f"../EGA.{object_of_file}.json", 
                           json_data_file = testing_file)

-/- Validation error found at file: ArrayAssay-invalid-1.json
	Cause: 130 is not of type 'string'
-/- Validation error found at file: ArrayAssay-invalid-2.json
	Cause: 'MD20' is not one of ['MD5 [NCIT:C171276]', 'SHA-256 [NCIT:C80226]']
-/- Validation error found at file: ArrayAssay-invalid-3.json
	Cause: '2121-13-32' does not match '^(19|20)[0-9]{2}-(0[0-9]|1[0-2])-([012][0-9]|3[01])$'
- Correctly validated file: ArrayAssay-valid-1.json

-/- Validation error found at file: ArrayExperiment-invalid-1.json
	Cause: 'EGAE0000000001' does not match '^EGAE[0-9]{11}$'
-/- Validation error found at file: ArrayExperiment-invalid-2.json
	Cause: 'EGAE00000000001' does not match '^EGAN[0-9]{11}$'
-/- Validation error found at file: ArrayExperiment-invalid-3.json
	Cause: 'protocol_ontology' is a required property
-/- Validation error found at file: ArrayExperiment-invalid-4.json
	Cause: 'c6779ec2960296ed9a04f08d67f64423' does not match '^[A-Fa-f0-9]{64}$'
- Correctly validated file: ArrayExperiment